In [ ]:
def SLBL(grid_dem,grid_mask,tol,maxt,maxv,z_min,planes=None):
	# initilizes thickness and difference grids
	s=grid_dem.shape
	grid_thickn = np.zeros(s) # This just creates an array of zer0s with the shape equal to the grid_dem
	grid_diff = np.ones(s) # This just creates an array of ones (1) with the shape equal to the grid_dem

	# Creates a matrice to store the values of the neighbouring cells in the previous iteration
	mat_neigh = np.zeros(s)
	mat_neigh = np.expand_dims(mat_neigh,axis=2) # just changes the dimensionality of the array. no changes in the data, just the shape of the array. 

	# NOTE: nb_neigh is initialized at the main script
	# nb_neigh is 4, if the method selected uses "4 neighbors, average" or "4 neighbors, min/max" 
	if nb_neigh ==4:
		mat_neigh = np.tile(mat_neigh,(1,1,4))
	else: 
		mat_neigh = np.tile(mat_neigh,(1,1,8))

	# Creates a matrice where the proposed value and previous value are stored for comparison
	mat_comp = np.zeros(s)
	mat_comp = np.expand_dims(mat_comp,axis=2)
	mat_comp = np.tile(mat_comp,(1,1,2))

	# Initiate the slbl grid (altitude)
	grid_slbl = deepcopy(grid_dem)

	nb_iter = 0
	volume = 0.

	if np.isfinite(maxt):
		grid_maxt = grid_dem - maxt

	# The SLBL strarts here
	while np.amax(grid_diff)>stop and volume<maxv:
		nb_iter=nb_iter+1
		grid_thickn_prev = deepcopy(grid_thickn)
		grid_slbl_prev = deepcopy(grid_slbl)
		
		# writes the values of the neighbourings cells in the 3rd dimension of the matrix
		mat_neigh[:-1,:,0]=grid_slbl_prev[1:,:]
		mat_neigh[1:,:,1]=grid_slbl_prev[:-1,:]
		mat_neigh[:,:-1,2]=grid_slbl_prev[:,1:]
		mat_neigh[:,1:,3]=grid_slbl_prev[:,:-1]
		
		# diagonals
		if nb_neigh ==8:
			mat_neigh[:-1,:-1,4]=grid_slbl_prev[1:,1:]
			mat_neigh[:-1,1:,5]=grid_slbl_prev[1:,:-1]
			mat_neigh[1:,1:,6]=grid_slbl_prev[:-1,:-1]
			mat_neigh[1:,:-1,7]=grid_slbl_prev[:-1,1:]
		
		if criteria == 'minmax':
			mat_max=np.amax(mat_neigh,axis=2)
			mat_min=np.amin(mat_neigh,axis=2)
			mat_mean=(mat_max+mat_min)/2
		elif criteria == 'average':
			mat_mean=np.mean(mat_neigh,axis=2)
		mat_mean=mat_mean+tol
		# limit to the lower altitude around the polygon
		if np.isfinite(z_min):
			mat_mean=np.maximum(mat_mean,z_min)
		# limit to the maximum thickness
		if np.isfinite(maxt):
			mat_mean=np.maximum(mat_mean,grid_maxt)
		if not planes is None:
			mat_mean=np.maximum(mat_mean,planes)
		
		mat_comp[:,:,0]=mat_mean
		mat_comp[:,:,1]=grid_slbl
		
		# Check if the new value should be kept
		if inverse == 'true':
			grid_slbl=np.amax(mat_comp,axis=2)
		else:
			grid_slbl=np.amin(mat_comp,axis=2)
		
		# Replaces the values of the SLBL by the original values outside the masked area
		grid_slbl[~grid_mask]=grid_dem[~grid_mask]
		
		grid_thickn = np.absolute(grid_dem - grid_slbl)
		grid_diff = np.absolute(grid_thickn - grid_thickn_prev)
		
		volume = (np.sum(grid_thickn)*cellSize*cellSize)
		
		if nb_iter%100==0:
			str_message = '{} iterations. Max diff is {:.3e} m, max thickness is {:.3f} m and volume is {:.6f} million m3'.format(nb_iter,np.amax(grid_diff),np.amax(grid_thickn),volume/1000000)
			arcpy.AddMessage(str_message)
	# The SLBL is finished
	
	return grid_slbl, grid_thickn, nb_iter

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(12000, 16000))